<a href="https://colab.research.google.com/github/MEHARKhaoula/transformers-text-coherence/blob/main/RobertaSem_cv_Yahoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-662194ea-ec0e-b120-8269-84f5acbc57c1)


# 1. Setup

## 1.1. Using Colab GPU for Training


In [2]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In order for torch to use the GPU, we need to identify and specify the GPU as the device. Later, in our training loop, we will load data onto the device. 

In [3]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


## 1.2. Installing the Hugging Face Library


In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 106.2 MB/s eta 0:00:00


In [5]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Importing GCDC dataset



## 2.2. Parse

We'll use pandas to parse the "in-domain" training set and look at a few of its properties and data points.

In [6]:


# Load the dataset into a pandas dataframe.
df = pd.read_csv("/content/drive/My Drive/Yahoo_train.csv")
df['labelA'] = df['labelA'].astype(int)
df['labelA'] = df['labelA'] - 1

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

Number of training sentences: 1,000



,text_id,question_title,question,text,ratingA1,ratingA2,ratingA3,labelA,ratingM1,ratingM2,ratingM3,ratingM4,ratingM5,labelM
185,text1939PG,If is not race related???,Why I don't see you blaming and name calling t...,Although I respect your sentiment and applaud ...,3,3,2,2,2,2,1,2,3,2
70,text906PG,S&W 620 vs Ruger GP100?,I'm wanting to buy a birthday present for my w...,The choice of weapon is as personal as your ch...,3,2,3,2,3,2,1,2,2,2
415,text829NE,What is the most important but underreported s...,"Even in our 24/7, 500 channel universe, lots o...",our government (state and local ) now having t...,2,1,2,0,2,3,2,1,3,2
867,text1371T,What options are there for internet access on ...,Other than PAYING the ridiculous prices that m...,Forget about using the ships internet service....,3,3,3,2,3,2,2,3,2,3
502,text188SC,"God created hermaphrodite creatures, therefore...",Just to point out.. I'm heterosexual and happi...,"You may have put this as a flippant question, ...",2,2,1,0,1,2,3,1,3,2
844,text661FR,This question is for parents of teens and pret...,Does lack of respect from your teen or preteen...,"Good luck, honey. That is what a teenager does...",3,1,1,0,2,2,1,3,1,1
664,text1978T,Why do so many people from Alberta try to act ...,"Seriously, if you ever go to Calgary you'll se...",I beg to differ that if Albertans walked aroun...,1,2,1,0,2,2,1,2,2,1
615,text890T,Has anyone been to Italy?,I am going to Italy today and I am kind of sca...,"If you are going to the larger cities, than ye...",1,3,2,1,2,2,2,3,3,3
426,text817T,"udapest to Cluj -- I want to take a bus, need ...","I am leaving Monday and need to be in Cluj, Ro...",where are you? are you in budapest? so you wan...,1,2,3,1,2,2,1,1,1,1
979,text1849SC,What do you got to say about that !?,"While thousands saw Jesus and heard his voice,...",i also have some points\n\nGod is the ultimate...,2,3,2,2,3,2,2,2,3,3


The two properties we actually care about are the the `text` and its `labelA`, which is referred to as the "coherence class"



Let's extract the sentences and labels of our training set as numpy ndarrays.

In [7]:
# Get the lists of sentences and their labels.
documents = df.text.values
labels = df.labelA.values

# 3. Tokenization & Input Formatting

In this section, we'll transform our dataset into the format that Roberta can be trained on.

## 3.1. Roberta Tokenizer

In [8]:
from transformers import RobertaTokenizer

# Load the Roberta tokenizer.
print('Loading Roberta tokenizer...')
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)

Loading Roberta tokenizer...


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Let's apply the tokenizer to one sentence just to see the output.


In [9]:
# Print the original sentence.
print(' Original: ', documents[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(documents[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(documents[0])))

 Original:  There are millions of us that do care, that even though we didn't lose someone personally, the events of 9-11 touched & affected ppl all over the world. The events of that day and those who were lost haven't been forgotten, nor should they ever be. It is our responsibility and duty to strive to bring something good out of all the horrible events of that day. (& it began the moment the first plane struck, there were many acts of bravery, heroism, self-sacrifice, and kindness). A beautiful monument will be built in honor and rememberance of those lost - however that is not the most important thing to do. The most important way to remember and honor those lost is to improve security, raise the standards of training for those in airports, in security, change and modify high rise rescue policies, new communications between fire and police departments,,,,,it is thru these that those loved ones that were lost will be truly honored.
Tokenized:  ['There', 'Ġare', 'Ġmillions', 'Ġof',

## 3.2. Required Formatting

### Special Tokens


## 3.3. Tokenize Dataset

In [10]:
max_len = 0

# For every sentence...
for doc in documents:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(doc, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  386


Now we're ready to perform the real tokenization.

The `tokenizer.encode_plus` function combines multiple steps for us:

1. Split the sentence into tokens.
2. Add the special `[CLS]` and `[SEP]` tokens.
3. Map the tokens to their IDs.
4. Pad or truncate all sentences to the same length.
5. Create the attention masks which explicitly differentiate real tokens from `[PAD]` tokens.

The first four features are in `tokenizer.encode`, but I'm using `tokenizer.encode_plus` to get the fifth item (attention masks). Documentation is [here](https://huggingface.co/transformers/main_classes/tokenizer.html?highlight=encode_plus#transformers.PreTrainedTokenizer.encode_plus).


In [11]:
import torch
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for doc in documents:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        doc,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', documents[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  There are millions of us that do care, that even though we didn't lose someone personally, the events of 9-11 touched & affected ppl all over the world. The events of that day and those who were lost haven't been forgotten, nor should they ever be. It is our responsibility and duty to strive to bring something good out of all the horrible events of that day. (& it began the moment the first plane struck, there were many acts of bravery, heroism, self-sacrifice, and kindness). A beautiful monument will be built in honor and rememberance of those lost - however that is not the most important thing to do. The most important way to remember and honor those lost is to improve security, raise the standards of training for those in airports, in security, change and modify high rise rescue policies, new communications between fire and police departments,,,,,it is thru these that those loved ones that were lost will be truly honored.
Token IDs: tensor([    0,   970,    32,  2535,    

In [12]:
print('Original: ', documents[0])
print('Token IDs:', input_ids[0])

Original:  There are millions of us that do care, that even though we didn't lose someone personally, the events of 9-11 touched & affected ppl all over the world. The events of that day and those who were lost haven't been forgotten, nor should they ever be. It is our responsibility and duty to strive to bring something good out of all the horrible events of that day. (& it began the moment the first plane struck, there were many acts of bravery, heroism, self-sacrifice, and kindness). A beautiful monument will be built in honor and rememberance of those lost - however that is not the most important thing to do. The most important way to remember and honor those lost is to improve security, raise the standards of training for those in airports, in security, change and modify high rise rescue policies, new communications between fire and police departments,,,,,it is thru these that those loved ones that were lost will be truly honored.
Token IDs: tensor([    0,   970,    32,  2535,    

In [13]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(input_ids, attention_masks, labels)

In [14]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning Roberta on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )


# 4. Train Our Classification Model

Now that our input data is properly formatted, it's time to fine tune the RoBERTa model. 

## 4.1. RobertaForSequenceClassification

In [15]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import json
from transformers import RobertaForSequenceClassification

class RobertaSem(nn.Module):

  def __init__(self):
    super(RobertaSem, self).__init__()
    self.Roberta_layer = RobertaForSequenceClassification.from_pretrained(
          "roberta-base", 
          num_labels = 3,  
          output_attentions = False,
          output_hidden_states = False,
      )

  def forward(self, input_ids, attention_mask, token_type_ids=None, labels=None):
    return self.Roberta_layer(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels=labels, return_dict=True)


In [16]:
from transformers import RobertaForSequenceClassification, AdamW, RobertaConfig

# Load RoBertForSequenceClassification, the pretrained Roberta model with a single 
# linear classification layer on top. 
model = RobertaSem()
# Tell pytorch to run this model on the GPU.
model = model.to(device)

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

In [17]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The Roberta model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The Roberta model has 201 different named parameters.

==== Embedding Layer ====

Roberta_layer.roberta.embeddings.word_embeddings.weight (50265, 768)
Roberta_layer.roberta.embeddings.position_embeddings.weight   (514, 768)
Roberta_layer.roberta.embeddings.token_type_embeddings.weight     (1, 768)
Roberta_layer.roberta.embeddings.LayerNorm.weight             (768,)
Roberta_layer.roberta.embeddings.LayerNorm.bias               (768,)

==== First Transformer ====

Roberta_layer.roberta.encoder.layer.0.attention.self.query.weight   (768, 768)
Roberta_layer.roberta.encoder.layer.0.attention.self.query.bias       (768,)
Roberta_layer.roberta.encoder.layer.0.attention.self.key.weight   (768, 768)
Roberta_layer.roberta.encoder.layer.0.attention.self.key.bias       (768,)
Roberta_layer.roberta.encoder.layer.0.attention.self.value.weight   (768, 768)
Roberta_layer.roberta.encoder.layer.0.attention.self.value.bias       (768,)
Roberta_layer.roberta.encoder.layer.0.attention.output.dense.weight  

## 4.2. Optimizer & Learning Rate Scheduler

In [18]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [19]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The Roberta authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 3

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

Helper function for formatting elapsed times as `hh:mm:ss`


In [20]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


We're ready to kick off the training!

In [21]:
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits =10, shuffle = False)

In [22]:
from sklearn.metrics import accuracy_score
from sklearn import metrics

def eval_docs(prediction_dataloader):
  # Prediction on test set
  # Put model in evaluation mode
  model.eval()

  # Tracking variables 
  predictions , true_labels = [], []

  # Predict 
  for batch in prediction_dataloader:
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
    
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels = batch
      
      # Telling the model not to compute or store gradients, saving memory and 
      # speeding up prediction
      with torch.no_grad():
          # Forward pass, calculate logit predictions.
          result = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask,
                        )
      results = F.softmax(result.logits, dim=0)

      logits = results

      # Move logits and labels to CPU
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()
      # Store predictions and true labels
      predictions.extend(list(np.argmax(logits, axis=1)))
      true_labels.extend(label_ids)

  matrix = metrics.confusion_matrix(true_labels, predictions, labels=[0, 1, 2])
  print(matrix)
  sum = np.sum(matrix)
  acc_low = (matrix[0][0] + matrix[1][1] + matrix[1][2] + matrix[2][1] + matrix [2][2])/(sum) #exactitude relative à la classe low
  acc_medium = (matrix[0][0] + matrix[1][1] + matrix[0][2] + matrix[2][0] + matrix [2][2])/(sum) #exactitude relative à la classe medium
  acc_high = (matrix[0][0] + matrix[1][1] + matrix[0][1] + matrix[1][0] + matrix [2][2])/(sum) #exactitude relative à la classe high

  accuracy = accuracy_score(predictions, true_labels)
  print("Accuracy :")
  print(accuracy)
  print("Accuracy low :")
  print(acc_low)  
  print("Accuracy medium :")
  print(acc_medium) 
  print("Accuracy high :")
  print(acc_high) 
  print("Average accuracy :")
  print((acc_low + acc_medium + acc_high)/3) 
  print("Classification report :")
  print(metrics.classification_report(true_labels, predictions, labels=[0, 1, 2], zero_division=1))

  print('    DONE.')
  return accuracy

In [23]:
import random
import numpy as np
import torch.nn.functional as F
import torch.utils.data as d
import sys


# Set the seed value all over the place to make this reproducible.
seed_val = 42 
best_test_acc = 0

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    CUDA_LAUNCH_BLOCKING=1
    # Put the model into training mode
    model.train()

    for fold, (train, test) in enumerate(kfold.split(train_dataset, labels)):
      train_subsampler = d.SubsetRandomSampler(train)
      test_subsampler = d.SubsetRandomSampler(test)

      trainloader = DataLoader(
                train_dataset, 
                batch_size=batch_size, sampler=train_subsampler)
      testloader = torch.utils.data.DataLoader(
                train_dataset,
                batch_size=batch_size, sampler=test_subsampler)
  # For each batch of training data...
      for step, batch in enumerate(trainloader):

          # Progress update every 40 batches.
          if step % 40 == 0 and not step == 0:
              # Calculate elapsed time in minutes.
              elapsed = format_time(time.time() - t0)
              
              # Report progress.
              print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(trainloader), elapsed))

          # Unpack this training batch from our dataloader. 
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_labels = batch[2].to(device)

          # Always clear any previously calculated gradients before performing a
          # backward pass.
          model.zero_grad()        

          # Perform a forward pass (evaluate the model on this training batch).
          result = model(b_input_ids, 
                      token_type_ids=None, 
                      attention_mask=b_input_mask, 
                      labels=b_labels,
                      )
          results = F.softmax(result.logits, dim=0)
          loss = result.loss
          logits = results

          # Accumulate the training loss over all of the batches so that we can
          # calculate the average loss at the end.
          total_train_loss += loss.item()

          # Perform a backward pass to calculate the gradients.
          loss.backward()

          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

          # Update parameters and take a step using the computed gradient.
          optimizer.step()

          # Update the learning rate.
          scheduler.step()

      # Calculate the average loss over all of the batches.
      avg_train_loss = total_train_loss / len(trainloader)            
      
      # Measure how long this epoch took.
      training_time = format_time(time.time() - t0)

      print("")
      print("  Average training loss: {0:.2f}".format(avg_train_loss))
      print("  Training epcoh took: {:}".format(training_time))

          # Record all statistics from this epoch.
      training_stats.append(
          {
              'epoch': epoch_i + 1,
              'Training Loss': avg_train_loss,
              'Training Time': training_time,
          }
      )
      test_accuracy = eval_docs(testloader)
      if test_accuracy > best_test_acc:
        best_test_acc = test_accuracy

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

print("==================== BEST TEST ACCURACY =================================")
print(best_test_acc)


======== Epoch 1 / 3 ========
Training...

  Average training loss: 1.03
  Training epcoh took: 0:00:41
[[32  5  8]
 [ 3  7  8]
 [ 4 17 16]]
Accuracy :
0.55
Accuracy low :
0.8
Accuracy medium :
0.67
Accuracy high :
0.63
Average accuracy :
0.7000000000000001
Classification report :
              precision    recall  f1-score   support

           0       0.82      0.71      0.76        45
           1       0.24      0.39      0.30        18
           2       0.50      0.43      0.46        37

    accuracy                           0.55       100
   macro avg       0.52      0.51      0.51       100
weighted avg       0.60      0.55      0.57       100

    DONE.

  Average training loss: 1.92
  Training epcoh took: 0:01:21
[[32  3 10]
 [ 7  6  5]
 [ 8  9 20]]
Accuracy :
0.58
Accuracy low :
0.72
Accuracy medium :
0.76
Accuracy high :
0.68
Average accuracy :
0.7200000000000001
Classification report :
              precision    recall  f1-score   support

           0       0.68      0

Let's view the summary of the training process.

In [24]:
import pandas as pd

# Display floats with two decimal places.
pd.set_option('precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

,Training Loss,Training Time
epoch,,
1,1.03,0:00:41
1,1.92,0:01:21
1,2.69,0:02:00
1,3.37,0:02:39
1,4.06,0:03:18
1,4.75,0:03:57
1,5.44,0:04:37
1,6.15,0:05:16
1,6.85,0:05:55


# Save the model

In [25]:
import pickle

torch.save(model.state_dict(), './RobertaSem.pt')
print('saved model ')

saved model 


In [26]:
# Fonction pour appliquer du preprocessing sur les données à fournir au modèle Roberta
# avant de faire des prédictions (par prédiction je veux dire : donner un document en entrée et recevoir la classe prédite {Low, Medium, High})
# NOTE : il faut d'abord instancier le tokenizer + importer les librairies nécessaires (exécuter les cellules concernées dessus)
def preprocess_data_Roberta(text):
    input_ids = []
    attention_masks = []
    
    encoded_dict = tokenizer.encode_plus(
                        text,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])
    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    
    sample = TensorDataset(input_ids, attention_masks)
    sample_loader = DataLoader(
            sample,
            sampler = RandomSampler(sample),
            batch_size = 1
        )
    return sample_loader

In [27]:
model = torch.load(open('./RobertaSem.pt', 'rb'), map_location=torch.device('0'))
model.eval()
sample_loader = preprocess_data_Roberta(sample)
for s in sample_loader: 
    sample = tuple(t for t in s)
b_input_ids, b_input_mask = sample
pred = model.forward(b_input_ids=b_input_ids, b_input_mask=b_input_mask)
result = F.softmax(pred.logits, dim=1)
result = result.cpu().data.numpy()
argmax = list(np.argmax(result, axis=1))
score = json.dumps(argmax[0], cls=NumpyArrayEncoder)
print(f'score : {score}')

RuntimeError: ignored

In [ ]:
import matplotlib.pyplot as plt
% matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")

# Label the plot.
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3])

plt.show()